<a href="https://colab.research.google.com/github/Valerie-Fan/python-data-analysis-and-machine-learning/blob/main/sales-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import the datasets

In [ ]:
import zipfile
import os
import shutil
import pandas as pd
import numpy as np

# Unzip the file
with zipfile.ZipFile('/content/store_sale_prediction.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
# List the files in the subdirectory
data_dir = '/content/data/store_sale_prediction'
data_files = os.listdir(data_dir)
data_files

['validations.csv',
 'oil.csv',
 'transactions.csv',
 'dataset.csv',
 'holidays_events.csv',
 'stores.csv']

In [ ]:
import pandas as pd

# Load each file into a pandas DataFrame
data_path = lambda file: os.path.join(data_dir, file)

df = pd.read_csv(data_path('dataset.csv'))
holidays_events_df = pd.read_csv(data_path('holidays_events.csv'))
oil_df = pd.read_csv(data_path('oil.csv'))
stores_df = pd.read_csv(data_path('stores.csv'))
transactions_df = pd.read_csv(data_path('transactions.csv'))
validations_df = pd.read_csv(data_path('validations.csv'))

In [ ]:
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
holidays_events_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [ ]:
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [ ]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
validations_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


## Data Preprocessing

### Merging

In [ ]:
df = df.merge(stores_df ,how = 'left', on = 'store_nbr')

df = df.merge(holidays_events_df,how = 'left', on = ['date'])

df = df.merge(oil_df,how = 'left', on = 'date')

df

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57


### Handling missing values

In [ ]:
df.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
dcoilwtico      955152
dtype: int64

In [ ]:
# Apart from holidays all are working days
df['type_y'].fillna('working_day',inplace=True)

# All the NaN value in 'locale' column are working days hence it is National
df['locale'].fillna('National', inplace=True)

# All the NaN value in 'locale' column are working days hence it is National
df['description'].fillna('Unkown', inplace=True)

# Filling the blanks with the value before nan
df['dcoilwtico'].fillna(method = 'ffill', inplace=True)

# Filling the blanks with the value after nan
df['dcoilwtico'].fillna(method = 'bfill', inplace=True)

# All the NaN values are working days hence not tranferred
df['transferred'].fillna(False, inplace=True)

In [ ]:
# checking missgin  values
df.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
type_y               0
locale               0
locale_name    2551824
description          0
transferred          0
dcoilwtico           0
dtype: int64

## Feature Engineering

In [ ]:
# convert date column to datetime
df_copy = df.copy()
df_copy['date'] = pd.to_datetime(df_copy['date'], format='%Y-%m-%d')

# Extract datetime features
df_copy['month'] = df_copy['date'].dt.month
df_copy['day_of_month'] = df_copy['date'].dt.day
df_copy['day_of_year'] = df_copy['date'].dt.dayofyear
df_copy['week_of_year'] = df_copy['date'].dt.isocalendar().week
df_copy['day_of_week'] = df_copy['date'].dt.dayofweek
df_copy['year'] = df_copy['date'].dt.year

# fill NaN values in type column with 'Work Day'
df_copy['type_y'].fillna('Work Day', inplace=True)

# create binary column for holidays and work days
df_copy['is_holiday'] = np.where((df_copy['type_y'] != 'Work Day') & (df_copy['transferred'] == False), 1, 0)
df_copy['is_work_day'] = np.where((df_copy['type_y'] == 'Work Day') | (df_copy['transferred'] == True), 1, 0)

# group by year, month, and is_holiday columns to get holiday count per month
holidays_per_month = df_copy.groupby(['year', 'month', 'is_holiday'], as_index=False)['date'].count()
holidays_per_month.rename(columns={'date': 'holiday_count'}, inplace=True)
holidays_per_month = holidays_per_month[holidays_per_month['is_holiday'] == 1]

# merge holiday count per month with original dataframe
df_copy = pd.merge(df_copy, holidays_per_month[['year', 'month', 'holiday_count']], on=['year', 'month'], how='left')

In [ ]:
# Convert date column to datetime format
df_copy['date'] = pd.to_datetime(df_copy['date'])

# Create a new column for day of the month
df_copy['day'] = df_copy['date'].dt.day

# Create a new column for whether the day is a pay day or not
df_copy['pay_day'] = ((df_copy['day'] == 15) | (df_copy['day'] == df_copy['date'].dt.daysinmonth)).astype(int)

# Create a new column for whether the previous day was a pay day or not
df_copy['prev_pay_day'] = df_copy['pay_day'].shift(1).fillna(0)

In [ ]:
# drop culomns that would not be used in the model
df_copy.drop(['locale_name','description'],axis=1,inplace=True)

In [ ]:
df_copy.shape

(3054348, 26)

In [ ]:
# save df_copy
# df_copy.to_csv('df_copy.csv',index=False)

## Prepare Training Data

In [ ]:
df_copy.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,day_of_year,week_of_year,day_of_week,year,is_holiday,is_work_day,holiday_count,day,pay_day,prev_pay_day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,1,1,1,2013,1,0,51678,1,0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,1,1,1,2013,1,0,51678,1,0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,1,1,1,2013,1,0,51678,1,0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,1,1,1,2013,1,0,51678,1,0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,1,1,1,2013,1,0,51678,1,0,0.0


In [ ]:
# Drop the original 'date' column
df_copy.drop('date', axis=1, inplace=True)

In [ ]:
# Assuming 'target' is your target variable
X = df_copy.drop(columns=['sales'])
y = df_copy['sales']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['number']).columns

In [ ]:
# scaling numeric features
from sklearn.preprocessing import MinMaxScaler

# Scale numeric features
scaler = MinMaxScaler().fit(X_train[numeric_cols])

X_train_scaled = pd.DataFrame(scaler.transform(X_train[numeric_cols]), columns=numeric_cols, index=X_train.index)
X_val_scaled = pd.DataFrame(scaler.transform(X_val[numeric_cols]), columns=numeric_cols, index=X_val.index)

X_train = X_train.drop(columns=numeric_cols).join(X_train_scaled)
X_val = X_val.drop(columns=numeric_cols).join(X_val_scaled)

X.describe().loc[['min', 'max']]

,id,store_nbr,onpromotion,cluster,dcoilwtico,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_holiday,is_work_day,holiday_count,day,pay_day,prev_pay_day
min,0.0,1.0,0.0,1.0,26.19,1.0,1.0,1.0,1.0,0.0,2013.0,0.0,0.0,24948.0,1.0,0.0,0.0
max,3000887.0,54.0,741.0,17.0,110.62,12.0,31.0,366.0,53.0,6.0,2017.0,1.0,1.0,60588.0,31.0,1.0,1.0


In [ ]:
# One-Hot Encode categorical features
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(X_train[categorical_cols])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Encode training data
X_train_encoded = pd.DataFrame(encoder.transform(X_train[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols), index=X_train.index)
X_train = X_train.drop(columns=categorical_cols).join(X_train_encoded)

In [ ]:
# Encode validation data
X_val_encoded = pd.DataFrame(encoder.transform(X_val[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols), index=X_val.index)
X_val = X_val.drop(columns=categorical_cols).join(X_val_encoded)

In [ ]:
X_train.head()

,transferred,id,store_nbr,onpromotion,cluster,dcoilwtico,month,day_of_month,day_of_year,week_of_year,...,type_y_Additional,type_y_Bridge,type_y_Event,type_y_Holiday,type_y_Transfer,type_y_Work Day,type_y_working_day,locale_Local,locale_National,locale_Regional
580605,False,0.191102,0.075472,0.000000,0.1875,0.791662,0.909091,0.566667,0.879452,0.884615,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
457845,False,0.150195,0.094340,0.000000,0.7500,0.962809,0.727273,0.300000,0.690411,0.692308,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2585113,False,0.846604,0.773585,0.011019,0.0625,0.226223,0.909091,0.933333,0.912329,0.903846,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1839313,False,0.604610,0.301887,0.000000,0.6875,0.250030,0.818182,0.566667,0.794521,0.788462,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2829206,False,0.926858,0.075472,0.012397,0.1875,0.319792,0.272727,0.433333,0.282192,0.269231,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Tranining

In [ ]:
# extract sample data
X_train = X_train.sample(frac=0.001, random_state=42)
y_train = y_train.loc[X_train.index]

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

reg = LinearRegression().fit(X_train, y_train)

val_preds=reg.predict(X_val)

#The val_predictions have neagtive values , replacing them with zero
new_list = [0 if i < 0 else i for i in val_preds]

mean_squared_log_error(y_val, new_list, squared=False)

2.666724052537312

### KNN

In [ ]:
# knn
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_train)

val_preds=knn.predict(X_val)

mean_squared_log_error(y_val, val_preds, squared=False)

3.004247078034128

### Decision Tree

In [ ]:
# dicision tree
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor().fit(X_train, y_train)

val_preds=dt.predict(X_val)

mean_squared_log_error(y_val, val_preds, squared=False)

1.6686541691236694

### Random Forest

In [ ]:
# random forest
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor().fit(X_train, y_train)

val_preds=rf.predict(X_val)

mean_squared_log_error(y_val, val_preds, squared=False)

1.564035848388946

## Model Fine-tuning

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV

# # Define the Random Forest model
# rf = RandomForestRegressor()

# # Grid search parameters
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Setup the grid search
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')

# # Fit the grid search to the training data
# grid_search.fit(X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Score:", best_score)

## Prediction

In [ ]:
# Predicting validation.csv
test_df = pd.read_csv('/content/data/store_sale_prediction/validations.csv')

test_df['date'] = pd.to_datetime(test_df['date'])

test_df = test_df.merge(stores_df ,how = 'left', on = 'store_nbr')

test_df = test_df.merge(holidays_events_df,how = 'left', on = ['date'])

test_df = test_df.merge(oil_df,how = 'left', on = 'date')

In [ ]:
test_df.isna().sum()

id                 0
date               0
store_nbr          0
family             0
onpromotion        0
city               0
state              0
type_x             0
cluster            0
type_y         26730
locale         26730
locale_name    26730
description    26730
transferred    26730
dcoilwtico      7128
dtype: int64

In [ ]:
# Apart from holidays all are working days
test_df['type_y'].fillna('working_day',inplace=True)

# All the NaN value in 'locale' column are working days hence it is National
test_df['locale'].fillna('National', inplace=True)

# All the NaN value in 'locale' column are working days hence it is National
test_df['description'].fillna('Unkown', inplace=True)

# Filling the blanks with the value before nan
test_df['dcoilwtico'].fillna(method = 'ffill', inplace=True)

# Filling the blanks with the value after nan
test_df['dcoilwtico'].fillna(method = 'bfill', inplace=True)

# All the NaN values are working days hence not tranferred
test_df['transferred'].fillna(False, inplace=True)

In [ ]:
test_df.isna().sum()

id                 0
date               0
store_nbr          0
family             0
onpromotion        0
city               0
state              0
type_x             0
cluster            0
type_y             0
locale             0
locale_name    26730
description        0
transferred        0
dcoilwtico         0
dtype: int64

In [ ]:
# feature engineering
# convert date column to datetime
df_copy = test_df.copy()
df_copy['date'] = pd.to_datetime(df_copy['date'], format='%Y-%m-%d')

# Extract datetime features
df_copy['month'] = df_copy['date'].dt.month
df_copy['day_of_month'] = df_copy['date'].dt.day
df_copy['day_of_year'] = df_copy['date'].dt.dayofyear
df_copy['week_of_year'] = df_copy['date'].dt.isocalendar().week
df_copy['day_of_week'] = df_copy['date'].dt.dayofweek
df_copy['year'] = df_copy['date'].dt.year

# fill NaN values in type column with 'Work Day'
df_copy['type_y'].fillna('Work Day', inplace=True)

# create binary column for holidays and work days
df_copy['is_holiday'] = np.where((df_copy['type_y'] != 'Work Day') & (df_copy['transferred'] == False), 1, 0)
df_copy['is_work_day'] = np.where((df_copy['type_y'] == 'Work Day') | (df_copy['transferred'] == True), 1, 0)

# group by year, month, and is_holiday columns to get holiday count per month
holidays_per_month = df_copy.groupby(['year', 'month', 'is_holiday'], as_index=False)['date'].count()
holidays_per_month.rename(columns={'date': 'holiday_count'}, inplace=True)
holidays_per_month = holidays_per_month[holidays_per_month['is_holiday'] == 1]

# merge holiday count per month with original dataframe
df_copy = pd.merge(df_copy, holidays_per_month[['year', 'month', 'holiday_count']], on=['year', 'month'], how='left')

In [ ]:
# Convert date column to datetime format
df_copy['date'] = pd.to_datetime(df_copy['date'])

# Create a new column for day of the month
df_copy['day'] = df_copy['date'].dt.day

# Create a new column for whether the day is a pay day or not
df_copy['pay_day'] = ((df_copy['day'] == 15) | (df_copy['day'] == df_copy['date'].dt.daysinmonth)).astype(int)

# Create a new column for whether the previous day was a pay day or not
df_copy['prev_pay_day'] = df_copy['pay_day'].shift(1).fillna(0)

In [ ]:
# drop culomns that would not be used in the model
df_copy.drop(['locale_name','description'],axis=1,inplace=True)

In [ ]:
numeric_cols = df_copy.select_dtypes(include=['number']).columns
categorical_cols = df_copy.select_dtypes(include=['object']).columns

In [ ]:
# scaling numeric features
from sklearn.preprocessing import MinMaxScaler

# Scale numeric features
scaler = MinMaxScaler().fit(df_copy[numeric_cols])

df_copy_scaled = pd.DataFrame(scaler.transform(df_copy[numeric_cols]), columns=numeric_cols, index=df_copy.index)

df_copy = df_copy.drop(columns=numeric_cols).join(df_copy_scaled)

In [ ]:
# One-Hot Encode categorical features
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(df_copy[categorical_cols])

# Encode testing data
df_copy_encoded = pd.DataFrame(encoder.transform(df_copy[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols), index=df_copy.index)
df_copy = df_copy.drop(columns=categorical_cols).join(df_copy_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
df_copy.drop('date', axis=1, inplace=True)

In [ ]:
# Best model
best_model = RandomForestRegressor(max_depth=40, min_samples_leaf=1, min_samples_split=2)

X_train.drop('type_y_Event',axis=1,inplace=True)
X_train.drop('locale_Regional',axis=1,inplace=True)
X_train.drop('type_y_Bridge',axis=1,inplace=True)
X_train.drop('type_y_Work Day',axis=1,inplace=True)
X_train.drop('type_y_Additional',axis=1,inplace=True)
X_train.drop('type_y_Transfer',axis=1,inplace=True)


best_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=40)

In [ ]:
missing_features = set(best_model.feature_names_in_) - set(df_copy.columns)
if missing_features:
    raise ValueError(f"Missing features in df_copy: {missing_features}")

# predict
y_pred = best_model.predict(df_copy)

In [ ]:
# save result
result = pd.DataFrame({'id': test_df['id'], 'sales': y_pred})
result.to_csv('result4.csv', index=False)